# **Analisis de sentimientos de comentarios de películas**

### Integrantes:
* Ángela Vargas
* Juan Martin Santos
* Daniel Osorio

## **Caso**
Análisis de sentimientos de películas. Este proyecto tiene comentarios de películas
en español, que deben ser clasificadas en las categorías de positivo, negativo. Esto por medio de técnicas para el procesamiento de lenguaje natural.

## **Entendimiento de los datos**
Primero vamos a cargar los datos en un DataFrame para luego revisar la calidad de estos en cuanto a completitud, consistencia y unicidad.

In [36]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import nltk
# Se descargan las stopwords (despues se usan para eliminar palabras que no aportan informacion)
nltk.download('stopwords')
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, precision_score, recall_score, f1_score, accuracy_score
# Este comando es requerido para que las visualizaciones se muestren en este notebook
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\osori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pd.set_option('display.max_columns', None) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar

In [3]:
# Lectura de datos en formato CSV
# Los datos son almacenados en memoria usando una estructura de datos de Pandas conocida como dataframe
df_reviews = pd.read_csv('./data/MovieReviews.csv')

In [4]:
# Muestra el numero de filas y columnas del dataframe
df_reviews.shape

(5000, 3)

In [5]:
# Muestra de 5 filas aleatorias del dataframe
df_reviews.sample(5)

,Unnamed: 0,review_es,sentimiento
3548,3548,No hace falta decir que se debe hacer un Módic...,negativo
2087,2087,"""Ven a deshacerme"" parece obtener muchas opini...",positivo
4010,4010,Directamente al video y con buena razón.Es com...,negativo
698,698,Esta es quizás la serie de televisión con el m...,positivo
646,646,La película es buena y creo que Tiffany Amber ...,positivo


In [6]:
# Muestra el tipo de datos de cada columna
df_reviews.dtypes

Unnamed: 0      int64
review_es      object
sentimiento    object
dtype: object

In [7]:
# Muestra los diferentes valores de la columna sentimiento
df_reviews['sentimiento'].value_counts()

positivo    2500
negativo    2500
Name: sentimiento, dtype: int64

Como podemos observar, tenemos 5000 registros (reviews) de los cuales la mitad corresponden a reviews positivas y la otra mitad a reviews negativas. Existe consistencia en los valores de la columna sentimiento. Como existen 2500 positivos y 2500 negativos, podemos decir que la completitud de los datos para la columna sentimiento es del 100%, vamos a revisar ahora la completitud para la columna *review_es*.

In [8]:
# Muestra la cantidad de valores nulos para la columna review_es
df_reviews['review_es'].isnull().sum()

0

Como podemos observar, tenemos una completitud del 100% en la columna review_es, por lo que no es necesario realizar ningún tipo de limpieza de datos.
## **Preprocesamiento**


Como libreria para el procesamiento de lenguaje natural vamos a utilizar NLTK (Natural Language Tool Kit). Primero definimos un set de stopwords en español. Las stopwords son esas palabras que no aportan significado al texto para la tarea de clasificación. Estas palabras vacías pueden ser conectores propios del idioma, pronombres o puntos y comas. Algunas stopwords del español son por ejemplo las siguientes.

In [9]:
# Seteamos las stopwords en español
stop_words = set(stopwords.words('spanish'))
# Agregamos a las stopwords tambien los signos de puntuacion
# Se agrega una u a la izquierda para indicar que es un string unicode, sin embargo no es necesario
stop_words.extend([u'.', u'[', ']', u',', u';', u'', u')', u'),', u' ', u'('])
print(stop_words)

{'tuvieras', 'tendríais', 'tiene', 'eras', 'tuve', 'habré', 'tuvo', 'sentido', 'ya', 'por', 'tuviese', 'vuestro', 'estoy', 'fueras', 'sintiendo', 'hayas', 'ni', 'tengan', 'estuviésemos', 'sentidos', 'estuviste', 'estaría', 'estado', 'poco', 'tendríamos', 'nuestros', 'tendrían', 'estábamos', 'éramos', 'hayamos', 'tendrás', 'vuestros', 'porque', 'estuvisteis', 'mías', 'un', 'fueses', 'ante', 'tuyo', 'otros', 'suyas', 'hubieseis', 'estamos', 'estuvieseis', 'hubiéramos', 'tengo', 'mucho', 'siente', 'tengáis', 'seré', 'esta', 'fue', 'habiendo', 'habías', 'desde', 'hubiese', 'suyos', 'nada', 'será', 'fuéramos', 'tendréis', 'estemos', 'tened', 'contra', 'estaremos', 'estad', 'teníamos', 'muy', 'hubiste', 'seremos', 'teníais', 'uno', 'estuvieron', 'mi', 'han', 'estos', 'tendrías', 'estabas', 'unos', 'fuisteis', 'habrán', 'estando', 'cuando', 'hay', 'sí', 'una', 'míos', 'fuiste', 'ellos', 'vosotros', 'estaba', 'tenido', 'nuestras', 'tuvierais', 'tuyos', 'estuviese', 'esos', 'fueseis', 'sería', 

Luego se utiliza el Snowball stemming algorithm, este algoritmo lo que hace es transformar/reducir las palabras a su forma base, aqui hacemos un ejemplo con la palabra 'corriendo' y 'correr'

In [27]:
snowball_stemmer = SnowballStemmer('spanish')
snowball_stemmer.stem('corriendo')

'corr'

In [19]:
snowball_stemmer.stem('correr')

'corr'

Para tokenizar utilizamos la función *wordpunct_tokenize()* que tiene la libreria NLTK. Esta función, a diferencia de la función *word_tokenize()*, almacena los signos de puntuación como tokens diferentes. El *word.isapha()* es para revisar que la palabra no sea de una sola letra como por ejemplo 'y'. Luego se revisa que la palabra en minuscula no esté entre las stopwords antes de tokenizarla. A continuación un ejemplo de como funciona

In [20]:
# Ejemplo de como se tokenizan las palabras de una reseña
review_example = "Este es un ejemplo de una reseña en español."
review_tokens = [word.lower() for word in wordpunct_tokenize(review_example) if word.isalpha() and word.lower() not in stop_words]
print(review_tokens)

['ejemplo', 'reseña', 'español']


Ahora aplicamos esto para cada fila de la columna 'review_es'. Para esto definimos la funcion lamba a cada fila de la columna review utilizando el metodo apply. Se hace todo el proceso de tokenizacion mostrado anteriormente y se vuelve a juntar el texto con el metodo join(). El string resultante es ahora el nuevo valor para la fila en la columna 'review_es'.

In [28]:
df_reviews['review_es'] = df_reviews['review_es'].apply(lambda x: ' '.join([snowball_stemmer.stem(word.lower()) for word in wordpunct_tokenize(x) if (word.isalpha() and word.lower() not in stop_words)]))

In [29]:
df_reviews['review_es'].head(5)

0    si busc pelicul guerr tipic asi not aficion gu...
1    supong director pelicul luj sent busc abrig gr...
2    dificil cont pelicul estrop disfrut esper vien...
3    pelicul comienz lent estil vid wallac napalm a...
4    pelicul verdader accion maxim expresion mejor ...
Name: review_es, dtype: object

Usamos el *CountVectorizer()* que nos da la cuenta de cada de cada palabra para cada review, lo cual podemos usar para los modelos que crearemos a continuación.

In [33]:
# Extraer la cuenta de cada palabra para cada review
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_reviews['review_es'])
y = df_reviews['sentimiento']

In [34]:
# Dividimos los datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## **Modelos**

### **Multinomial Naive Bayes**

Escogimos este algoritmo porque trabaja muy bien con datos discretos, lo cual lo hace especialmente bueno para una tarea de clasificacion como si el comentario es positivo o negativo. El algoritmo realiza el calculo de la probabilidad de que cada palabra de una review pertenezca a una clase (positivo o negativo) y luego utiliza estas probabilidades con el teorema de Bayes para calcular la probabilidad de que una review pertenezca a una clase. El algoritmo asume que la probabilidad de una palabra es independiente de las otras, por lo cual es llamado 'naive' Bayes.

In [48]:
# Entrenar el modelo
multinomialNB = MultinomialNB()
multinomialNB.fit(X_train, y_train)

# Predicción
y_train_pred = multinomialNB.predict(X_train)
y_pred = multinomialNB.predict(X_test)

# Metricas de evaluacion
print('Exactitud sobre entrenamiento: %.2f' % accuracy_score(y_train, y_train_pred))
print('Exactitud sobre test: %.2f' % accuracy_score(y_test, y_pred))
print("Recall: {}".format(recall_score(y_test, y_pred, pos_label='positivo')))
print("Precisión: {}".format(precision_score(y_test, y_pred, pos_label='positivo')))
print("Puntuación F1: {}".format(f1_score(y_test, y_pred, pos_label='positivo')))

Exactitud sobre entrenamiento: 0.95
Exactitud sobre test: 0.82
Recall: 0.7650727650727651
Precisión: 0.8382687927107062
Puntuación F1: 0.8


In [49]:
# Matriz de confusion
confusion_matrix(y_test, y_pred)

array([[448,  71],
       [113, 368]], dtype=int64)